In [1]:
import boto3
import progressbar

In [2]:
source_env = "dev"
target_env = "prod"

source_bucket_name = f"heroes-dl-silver-{source_env}"
destination_bucket_name = f"heroes-dl-silver-{target_env}"
prefix = "direct_airbyte_syncs/sp_api/"
# Keyword to search for
keyword = "GET_V2_SELLER_PERFORMANCE_REPORT"

# Define ingestion dates to copy
p_ingestion_dt_list = [
    "2024-07-10",
]

In [3]:
s3 = boto3.Session(profile_name=f"data-{target_env}").client("s3")

In [4]:
def s3_items_from_table(bucket, prefix):
    paginator = s3.get_paginator("list_objects_v2")
    pages = paginator.paginate(Bucket=bucket, Prefix=prefix)
    objects = []

    for page in pages:
        for item in page["Contents"]:
            objects.append(item["Key"])

    return objects

In [5]:
def copy_s3_object(source_bucket, source_key, destination_bucket, destination_key):
    copy_source = {"Bucket": source_bucket, "Key": source_key}
    destination = {"Bucket": destination_bucket, "Key": destination_key}

    try:
        # will overwrite a file in destination if one shows up
        s3.copy_object(
            CopySource=copy_source, Bucket=destination["Bucket"], Key=destination["Key"]
        )
        print(
            f"S3 object copied from {source_bucket}/{source_key} to {destination_bucket}/{destination_key}"
        )
    except Exception as e:
        print(f"Error copying S3 object: {e}")

In [6]:
# Get all s3 objets for selected bucket and prefix
s3_keys = s3_items_from_table(bucket=source_bucket_name, prefix=prefix)

print(len(s3_keys))
for element in s3_keys[:5]:
    print(element)

6477
direct_airbyte_syncs/sp_api/p_account_id=AMZ_PETS/p_region_id=EU/p_marketplace_id=DE/p_entity_id=GET_FBA_INVENTORY_PLANNING_DATA/p_ingestion_dt=2024-02-21/1708525848588_0.jsonl
direct_airbyte_syncs/sp_api/p_account_id=AMZ_PETS/p_region_id=EU/p_marketplace_id=DE/p_entity_id=GET_FBA_INVENTORY_PLANNING_DATA/p_ingestion_dt=2024-02-22/1708563758592_0.jsonl
direct_airbyte_syncs/sp_api/p_account_id=AMZ_PETS/p_region_id=EU/p_marketplace_id=DE/p_entity_id=GET_FBA_INVENTORY_PLANNING_DATA/p_ingestion_dt=2024-02-22/1708607009014_0.jsonl
direct_airbyte_syncs/sp_api/p_account_id=AMZ_PETS/p_region_id=EU/p_marketplace_id=DE/p_entity_id=GET_FBA_INVENTORY_PLANNING_DATA/p_ingestion_dt=2024-02-22/1708615552978_0.jsonl
direct_airbyte_syncs/sp_api/p_account_id=AMZ_PETS/p_region_id=EU/p_marketplace_id=DE/p_entity_id=GET_FBA_INVENTORY_PLANNING_DATA/p_ingestion_dt=2024-02-23/1708650163957_0.jsonl


In [8]:
# Filter the keyword
s3_keys_keyword_filtered = []
for i in s3_keys:
    if keyword in i:
        s3_keys_keyword_filtered.append(i)

print(len(s3_keys_keyword_filtered))
for element in s3_keys_keyword_filtered[:5]:
    print(element)

54
direct_airbyte_syncs/sp_api/p_account_id=AMZ_PETS/p_region_id=EU/p_marketplace_id=UK/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720602855563_0.jsonl
direct_airbyte_syncs/sp_api/p_account_id=AMZ_PETS/p_region_id=NA/p_marketplace_id=US/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720602326928_0.jsonl
direct_airbyte_syncs/sp_api/p_account_id=BABY_UMA/p_region_id=AU/p_marketplace_id=AU/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720602554440_0.jsonl
direct_airbyte_syncs/sp_api/p_account_id=BABY_UMA/p_region_id=EU/p_marketplace_id=UK/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720602659791_0.jsonl
direct_airbyte_syncs/sp_api/p_account_id=BABY_UMA/p_region_id=JP/p_marketplace_id=JP/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720601908739_0.jsonl


In [9]:
# Filter only selected ingestion dates
s3_keys_final = []
for i in s3_keys_keyword_filtered:
    for j in p_ingestion_dt_list:
        if j in i:
            s3_keys_final.append(i)

print(len(s3_keys_final))
for element in s3_keys_final[:5]:
    print(element)

54
direct_airbyte_syncs/sp_api/p_account_id=AMZ_PETS/p_region_id=EU/p_marketplace_id=UK/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720602855563_0.jsonl
direct_airbyte_syncs/sp_api/p_account_id=AMZ_PETS/p_region_id=NA/p_marketplace_id=US/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720602326928_0.jsonl
direct_airbyte_syncs/sp_api/p_account_id=BABY_UMA/p_region_id=AU/p_marketplace_id=AU/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720602554440_0.jsonl
direct_airbyte_syncs/sp_api/p_account_id=BABY_UMA/p_region_id=EU/p_marketplace_id=UK/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720602659791_0.jsonl
direct_airbyte_syncs/sp_api/p_account_id=BABY_UMA/p_region_id=JP/p_marketplace_id=JP/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720601908739_0.jsonl


In [10]:
# Copy the files from source to target one by one
for i in progressbar.progressbar(s3_keys_final, redirect_stdout=True):
    source_object_key = i
    # destination_object_key = i.replace("p_ingestion_dt=2024-01-17", "p_ingestion_dt=2024-01-01")
    destination_object_key = source_object_key
    copy_s3_object(
        source_bucket_name,
        source_object_key,
        destination_bucket_name,
        destination_object_key,
    )

  0% (0 of 54) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--


S3 object copied from heroes-dl-silver-dev/direct_airbyte_syncs/sp_api/p_account_id=AMZ_PETS/p_region_id=EU/p_marketplace_id=UK/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720602855563_0.jsonl to heroes-dl-silver-prod/direct_airbyte_syncs/sp_api/p_account_id=AMZ_PETS/p_region_id=EU/p_marketplace_id=UK/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720602855563_0.jsonl


  1% (1 of 54) |                         | Elapsed Time: 0:00:00 ETA:   0:00:12


S3 object copied from heroes-dl-silver-dev/direct_airbyte_syncs/sp_api/p_account_id=AMZ_PETS/p_region_id=NA/p_marketplace_id=US/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720602326928_0.jsonl to heroes-dl-silver-prod/direct_airbyte_syncs/sp_api/p_account_id=AMZ_PETS/p_region_id=NA/p_marketplace_id=US/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720602326928_0.jsonl


  3% (2 of 54) |                         | Elapsed Time: 0:00:00 ETA:   0:00:11


...

 96% (52 of 54) |####################### | Elapsed Time: 0:00:04 ETA:   0:00:00


S3 object copied from heroes-dl-silver-dev/direct_airbyte_syncs/sp_api/p_account_id=TRUNKI_US/p_region_id=NA/p_marketplace_id=US/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720602025332_0.jsonl to heroes-dl-silver-prod/direct_airbyte_syncs/sp_api/p_account_id=TRUNKI_US/p_region_id=NA/p_marketplace_id=US/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720602025332_0.jsonl


 98% (53 of 54) |####################### | Elapsed Time: 0:00:04 ETA:   0:00:00


S3 object copied from heroes-dl-silver-dev/direct_airbyte_syncs/sp_api/p_account_id=TUTTI_BAMBINI/p_region_id=EU/p_marketplace_id=UK/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720602575762_0.jsonl to heroes-dl-silver-prod/direct_airbyte_syncs/sp_api/p_account_id=TUTTI_BAMBINI/p_region_id=EU/p_marketplace_id=UK/p_entity_id=GET_V2_SELLER_PERFORMANCE_REPORT/p_ingestion_dt=2024-07-10/1720602575762_0.jsonl


100% (54 of 54) |########################| Elapsed Time: 0:00:05 Time:  0:00:05
